<a href="https://colab.research.google.com/github/boiBASH/Tolaram_Project/blob/main/tolaram_recomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import pandas
import pandas as pd

# Step 3: Load your dataset (replace with the actual path in your Drive)
file_path = "/content/drive/MyDrive/Data Analysis - Sample File.csv"
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-ef979cc6f3f4>:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [2]:
df

,Brand,Designation,SKU_Code,Salesman_Code,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value
0,1,FIELD EXECUTIVE,10002778,807259,8060733751,11432770,05/08/2024 00:00,3.0,"46,500"
1,1,FIELD EXECUTIVE,10002778,807881,9016204667,11565436,26/08/2024 00:00,1.0,"15,500"
2,1,FIELD EXECUTIVE,10002778,807881,9016204667,11727113,17/09/2024 00:00,1.0,"15,500"
3,1,FIELD EXECUTIVE,10002778,807881,8060359775,11756045,20/09/2024 00:00,1.0,"15,500"
4,1,FIELD EXECUTIVE,10002778,807881,9075013565,11585122,29/08/2024 00:00,1.0,"15,500"
...,...,...,...,...,...,...,...,...,...
108820,9,VAN SALES EXECUTIVE,VOB4500_,810667,8101599308,12090661,01/11/2024 00:00,1.0,"56,800"
108821,9,VAN SALES EXECUTIVE,VOB4500_,810667,8139407516,12282897,22/11/2024 00:00,1.0,"33,000"
108822,9,VAN SALES EXECUTIVE,VOB4500_,810667,8060468927,12067581,30/10/2024 00:00,4.0,"201,600"
108823,9,VAN SALES EXECUTIVE,VOB4500_,810667,8055148535,12032444,25/10/2024 00:00,1.0,"50,400"


In [3]:
# Step 1: Convert Delivered_date to datetime
df['Delivered_date'] = pd.to_datetime(df['Delivered_date'], errors='coerce', dayfirst=True)
#df['Delivered_date'] = df['Delivered_date'].dt.strftime('%Y-%m-%d')
#df['Delivered_date'] = pd.to_datetime(df['Delivered_date'])

In [4]:
print(df['Delivered_date'].isna().sum())

0


In [5]:

# Step 2: Clean and convert Redistribution Value (remove commas and convert to float)
df['Redistribution Value'] = df['Redistribution Value'].str.replace(',', '', regex=False).astype(float)

# Step 3: Fill missing 'Delivered Qty' with 0 (or could be dropped if preferred)
df['Delivered Qty'] = df['Delivered Qty'].fillna(0)


# Confirm changes
df.dtypes


,0
Brand,int64
Designation,object
SKU_Code,object
Salesman_Code,object
Customer_Phone,int64
Order_Id,int64
Delivered_date,datetime64[ns]
Delivered Qty,float64
Redistribution Value,float64


In [6]:
df

,Brand,Designation,SKU_Code,Salesman_Code,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value
0,1,FIELD EXECUTIVE,10002778,807259,8060733751,11432770,2024-08-05,3.0,46500.0
1,1,FIELD EXECUTIVE,10002778,807881,9016204667,11565436,2024-08-26,1.0,15500.0
2,1,FIELD EXECUTIVE,10002778,807881,9016204667,11727113,2024-09-17,1.0,15500.0
3,1,FIELD EXECUTIVE,10002778,807881,8060359775,11756045,2024-09-20,1.0,15500.0
4,1,FIELD EXECUTIVE,10002778,807881,9075013565,11585122,2024-08-29,1.0,15500.0
...,...,...,...,...,...,...,...,...,...
108820,9,VAN SALES EXECUTIVE,VOB4500_,810667,8101599308,12090661,2024-11-01,1.0,56800.0
108821,9,VAN SALES EXECUTIVE,VOB4500_,810667,8139407516,12282897,2024-11-22,1.0,33000.0
108822,9,VAN SALES EXECUTIVE,VOB4500_,810667,8060468927,12067581,2024-10-30,4.0,201600.0
108823,9,VAN SALES EXECUTIVE,VOB4500_,810667,8055148535,12032444,2024-10-25,1.0,50400.0


In [7]:
# 1. Total amount spent per purchase
df['Total_Amount_Spent'] = df['Redistribution Value'] * df['Delivered Qty']

# 2. Average product cost per SKU
sku_grouped = df.groupby('SKU_Code').agg({
    'Redistribution Value': 'sum',
    'Delivered Qty': 'sum'
}).rename(columns={
    'Redistribution Value': 'Total_Value',
    'Delivered Qty': 'Total_Quantity'
})
sku_grouped['Average_Product_Cost'] = sku_grouped['Total_Value'] / sku_grouped['Total_Quantity']

# Merge back to main DataFrame
df = df.merge(sku_grouped[['Average_Product_Cost']], on='SKU_Code', how='left')
df

,Brand,Designation,SKU_Code,Salesman_Code,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value,Total_Amount_Spent,Average_Product_Cost
0,1,FIELD EXECUTIVE,10002778,807259,8060733751,11432770,2024-08-05,3.0,46500.0,139500.0,15500.000000
1,1,FIELD EXECUTIVE,10002778,807881,9016204667,11565436,2024-08-26,1.0,15500.0,15500.0,15500.000000
2,1,FIELD EXECUTIVE,10002778,807881,9016204667,11727113,2024-09-17,1.0,15500.0,15500.0,15500.000000
3,1,FIELD EXECUTIVE,10002778,807881,8060359775,11756045,2024-09-20,1.0,15500.0,15500.0,15500.000000
4,1,FIELD EXECUTIVE,10002778,807881,9075013565,11585122,2024-08-29,1.0,15500.0,15500.0,15500.000000
...,...,...,...,...,...,...,...,...,...,...,...
108820,9,VAN SALES EXECUTIVE,VOB4500_,810667,8101599308,12090661,2024-11-01,1.0,56800.0,56800.0,49057.746991
108821,9,VAN SALES EXECUTIVE,VOB4500_,810667,8139407516,12282897,2024-11-22,1.0,33000.0,33000.0,49057.746991
108822,9,VAN SALES EXECUTIVE,VOB4500_,810667,8060468927,12067581,2024-10-30,4.0,201600.0,806400.0,49057.746991
108823,9,VAN SALES EXECUTIVE,VOB4500_,810667,8055148535,12032444,2024-10-25,1.0,50400.0,50400.0,49057.746991


In [8]:
def analyze_customer_purchases(customer_phone):
    customer_df = df[df['Customer_Phone'] == customer_phone].copy()

    if customer_df.empty:
        return f"No data found for customer phone: {customer_phone}"

    # Ensure date is sorted
    customer_df.sort_values('Delivered_date', inplace=True)

    # Add Month column
    customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

    # 1. List of SKUs bought
    skus_bought = customer_df['SKU_Code'].unique().tolist()

    # 2. Last purchase date for each SKU
    last_purchase = customer_df.groupby('SKU_Code')['Delivered_date'].max().dt.strftime('%Y-%m-%d').to_dict()

    # 3. Average quantity per month for each SKU
    monthly_qty = (
        customer_df.groupby(['SKU_Code', 'Month'])['Delivered Qty']
        .sum()
        .groupby('SKU_Code')
        .mean()
        .round(2)
        .to_dict()
    )

    # 4. Average interval (in months) between purchases
    avg_interval = {}
    for sku, group in customer_df.groupby('SKU_Code'):
        dates = group['Delivered_date'].drop_duplicates().sort_values()
        if len(dates) > 1:
            intervals = dates.diff().dropna().dt.days / 30.44
            avg_interval[sku] = round(intervals.mean(), 2)
        else:
            avg_interval[sku] = "One"

    # 5. Average monthly spend per SKU
    monthly_spend = (
        customer_df.groupby(['SKU_Code', 'Month'])['Total_Amount_Spent']
        .sum()
        .groupby('SKU_Code')
        .mean()
        .round(2)
        .to_dict()
    )

    # Final formatted output
    report = {
        'Customer Phone': customer_phone,
        'Total Unique SKUs Bought': len(skus_bought),
        'SKUs Bought': skus_bought,
        'Purchase Summary by SKU': {}
        }

    for sku in skus_bought:
        report['Purchase Summary by SKU'][sku] = {
            'Last Purchase Date': last_purchase.get(sku, 'N/A'),
            'Avg Monthly Quantity': monthly_qty.get(sku, 0),
            'Avg Purchase Interval (Months)': avg_interval.get(sku, 'N/A'),
            'Avg Monthly Spend': monthly_spend.get(sku, 0)
        }

    return report


In [9]:
sample_customer = df['Customer_Phone'].iloc[70]
result = analyze_customer_purchases(sample_customer)
from pprint import pprint
pprint(result)


{'Customer Phone': np.int64(8062671240),
 'Purchase Summary by SKU': {'10000001': {'Avg Monthly Quantity': 5.0,
                                          'Avg Monthly Spend': 317500.0,
                                          'Avg Purchase Interval (Months)': 'One',
                                          'Last Purchase Date': '2024-11-19'},
                             '10000002': {'Avg Monthly Quantity': 4.0,
                                          'Avg Monthly Spend': 137600.0,
                                          'Avg Purchase Interval (Months)': np.float64(0.41),
                                          'Last Purchase Date': '2024-12-07'},
                             '10000011': {'Avg Monthly Quantity': 1.0,
                                          'Avg Monthly Spend': 13500.0,
                                          'Avg Purchase Interval (Months)': np.float64(0.79),
                                          'Last Purchase Date': '2024-12-06'},
                    

In [10]:
#sample_customer = df['Customer_Phone'].iloc[70]
result = analyze_customer_purchases(7010009941)
from pprint import pprint
pprint(result)


{'Customer Phone': 7010009941,
 'Purchase Summary by SKU': {'10002849': {'Avg Monthly Quantity': 1.0,
                                          'Avg Monthly Spend': 18400.0,
                                          'Avg Purchase Interval (Months)': np.float64(1.61),
                                          'Last Purchase Date': '2024-10-03'},
                             '10002857': {'Avg Monthly Quantity': 1.0,
                                          'Avg Monthly Spend': 18916.67,
                                          'Avg Purchase Interval (Months)': np.float64(2.3),
                                          'Last Purchase Date': '2025-01-02'},
                             '10002866': {'Avg Monthly Quantity': 1.0,
                                          'Avg Monthly Spend': 18787.5,
                                          'Avg Purchase Interval (Months)': np.float64(1.46),
                                          'Last Purchase Date': '2024-12-26'},
                     

In [11]:
def predict_next_purchases(customer_phone):
    customer_df = df[df['Customer_Phone'] == customer_phone].copy()

    if customer_df.empty:
        return f"No data found for customer phone: {customer_phone}"

    customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

    # Purchase frequency
    freq = customer_df.groupby('SKU_Code')['Delivered_date'].nunique()

    # Total quantity & spend
    qty = customer_df.groupby('SKU_Code')['Delivered Qty'].sum()
    spend = customer_df.groupby('SKU_Code')['Total_Amount_Spent'].sum()

    # Time since last purchase
    last_purchase = customer_df.groupby('SKU_Code')['Delivered_date'].max()
    recency_days = (pd.Timestamp.today() - last_purchase).dt.days

    # Calculate score (simple heuristic)
    score_df = pd.DataFrame({
        'Frequency': freq,
        'Total_Quantity': qty,
        'Total_Spend': spend,
        'Recency': recency_days
    })

    # Normalize and compute probability score
    score_df['Frequency_Score'] = score_df['Frequency'] / score_df['Frequency'].sum()
    score_df['Recency_Score'] = 1 - (score_df['Recency'] / score_df['Recency'].max())
    score_df['Combined_Score'] = (
        0.5 * score_df['Frequency_Score'] +
        0.3 * (score_df['Total_Quantity'] / score_df['Total_Quantity'].sum()) +
        0.2 * score_df['Recency_Score']
    )

    # Convert to percentage
    score_df['Probability (%)'] = (score_df['Combined_Score'] / score_df['Combined_Score'].sum() * 100).round(2)

    # Predict expected quantity and spend based on monthly average
    avg_monthly_qty = customer_df.groupby(['SKU_Code', 'Month'])['Delivered Qty'].sum().groupby('SKU_Code').mean()
    avg_monthly_spend = customer_df.groupby(['SKU_Code', 'Month'])['Total_Amount_Spent'].sum().groupby('SKU_Code').mean()

    score_df['Expected Quantity'] = avg_monthly_qty.round(2)
    score_df['Expected Spend'] = avg_monthly_spend.round(2)

    # Suggest reward based on spend
    def suggest(row):
        if row['Expected Spend'] > 5000:
            return 'Offer Discount'
        elif row['Expected Quantity'] > 10:
            return 'Free Gift'
        else:
            return 'Thank You Message'

    score_df['Suggestion'] = score_df.apply(suggest, axis=1)

    # Sort and pick top 3
    top3 = score_df.sort_values('Probability (%)', ascending=False).head(3)

    return top3[['Expected Quantity', 'Expected Spend', 'Probability (%)', 'Suggestion']].reset_index()


In [14]:
#sample_customer = df['Customer_Phone'].iloc[70]
#result =
print(analyze_customer_purchases(7010009941))
predict_next_purchases(7010009941)
#from pprint import pprint
#pprint(result)

{'Customer Phone': 7010009941, 'Total Unique SKUs Bought': 4, 'SKUs Bought': ['10002849', '10002857', '10002866', '10002871'], 'Purchase Summary by SKU': {'10002849': {'Last Purchase Date': '2024-10-03', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': np.float64(1.61), 'Avg Monthly Spend': 18400.0}, '10002857': {'Last Purchase Date': '2025-01-02', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': np.float64(2.3), 'Avg Monthly Spend': 18916.67}, '10002866': {'Last Purchase Date': '2024-12-26', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': np.float64(1.46), 'Avg Monthly Spend': 18787.5}, '10002871': {'Last Purchase Date': '2024-11-28', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': 'One', 'Avg Monthly Spend': 8250.0}}}


,SKU_Code,Expected Quantity,Expected Spend,Probability (%),Suggestion
0,10002866,1.0,18787.50,39.22,Offer Discount
1,10002857,1.0,18916.67,32.02,Offer Discount
2,10002849,1.0,18400.00,15.70,Offer Discount


In [15]:
#sample_customer = df['Customer_Phone'].iloc[70]
#result =
print(analyze_customer_purchases(7010147753))
predict_next_purchases(7010147753)
#from pprint import pprint
#pprint(result)

{'Customer Phone': 7010147753, 'Total Unique SKUs Bought': 9, 'SKUs Bought': ['10002832', '10002835', '10002849', '10002866', '10002972', '10002805', 'CP25X75', '10002957', '10003395'], 'Purchase Summary by SKU': {'10002832': {'Last Purchase Date': '2024-10-03', 'Avg Monthly Quantity': 1.5, 'Avg Purchase Interval (Months)': np.float64(1.22), 'Avg Monthly Spend': 45600.0}, '10002835': {'Last Purchase Date': '2024-09-09', 'Avg Monthly Quantity': 2.0, 'Avg Purchase Interval (Months)': 'One', 'Avg Monthly Spend': 72000.0}, '10002849': {'Last Purchase Date': '2024-10-16', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': np.float64(1.15), 'Avg Monthly Spend': 18650.0}, '10002866': {'Last Purchase Date': '2024-09-11', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': 'One', 'Avg Monthly Spend': 18400.0}, '10002972': {'Last Purchase Date': '2024-09-18', 'Avg Monthly Quantity': 1.0, 'Avg Purchase Interval (Months)': 'One', 'Avg Monthly Spend': 80400.0}, '10002805': {'

,SKU_Code,Expected Quantity,Expected Spend,Probability (%),Suggestion
0,10002805,1.0,61333.33,24.10,Offer Discount
1,CP25X75,2.0,20000.00,14.00,Offer Discount
2,10002832,1.5,45600.00,12.56,Offer Discount
